# http://bit.ly/wsi-2017

Теперь посмотрим как семантические вектора помогают решать задачу Word Sense Induction - WSI.

Для начала скопируем из предыдущей тетрадки все полезное: загрузим модель и контексты, определим токенизатор и функцию векторного представления контекста.

In [2]:
import re
import numpy as np
from gensim.models import KeyedVectors
from pymystem3 import Mystem
import rl_wsd_labeled

mystem = Mystem()

In [3]:
w2v_model = KeyedVectors.load('sg_all_d256_m100.kv')

In [4]:
_, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))

In [5]:
def tokenize(s):
    return [t for t in mystem.lemmatize(s)
            if re.match('\w+$', t)]

def context_repr(context):
    left, _, right = context
    words = tokenize(left) + tokenize(right)
    return np.mean([w2v_model[w] for w in words if w in w2v_model],
                    axis=0)

context_repr(contexts[1][0])

array([ 0.00135083, -0.08833592,  0.02749542, -0.03430157,  0.04990597,
       -0.00351785, -0.0377895 ,  0.04325328,  0.03681576,  0.00777959,
       -0.09153348,  0.00741439, -0.00175712, -0.00361062, -0.06431929,
        0.03168813, -0.03431803, -0.04016968, -0.12728518, -0.05535114,
        0.05995908,  0.00678083,  0.01118196, -0.03073232,  0.00444479,
       -0.10487767, -0.02356375, -0.00680911,  0.05342769,  0.03441216,
        0.05997094, -0.0200747 ,  0.04236873, -0.03816266,  0.02926661,
       -0.00959332,  0.03278689,  0.01879168,  0.07646055, -0.01720432,
       -0.06004086, -0.02325079, -0.01187328,  0.04816246, -0.02895397,
       -0.02052972, -0.0224539 ,  0.03614068, -0.01640125, -0.00932809,
       -0.02643445,  0.10471622,  0.02175801,  0.10195923, -0.10238061,
       -0.01712064,  0.00831175,  0.00393068,  0.00358072,  0.02274694,
        0.0065462 , -0.05434006,  0.01133153, -0.03383037,  0.03999662,
        0.06762768, -0.0840156 ,  0.00743738, -0.05133355,  0.01

Подготовим данные для кластеризации:

In [6]:
xs = [ctx for ctx, _ in contexts]
ys = np.array([int(s) - 1 for _, s in contexts])
xs_vec = np.array([context_repr(ctx) for ctx in xs])

Попробуем использовать для кластеризации ``KMeans``:

In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6)
kmeans.fit(xs_vec)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

Результат кластеризации k-means это цетры кластеров:

In [8]:
kmeans.cluster_centers_

array([[-0.00694956, -0.04352317,  0.01485176, ..., -0.06973982,
        -0.06182503,  0.03310318],
       [ 0.02969682, -0.03872772,  0.03736759, ..., -0.00019917,
        -0.06179653,  0.06727503],
       [-0.03883437, -0.0733196 ,  0.00481461, ...,  0.0069336 ,
        -0.04950843, -0.00679021],
       [-0.00471178, -0.02607754,  0.01996918, ..., -0.0304148 ,
        -0.04241306,  0.02723102],
       [ 0.02350715, -0.13038456,  0.02251562, ..., -0.00828939,
        -0.03947693,  0.03027288],
       [ 0.00108585, -0.07813624,  0.02414697, ..., -0.02220607,
        -0.06131212,  0.03008327]], dtype=float32)

Также у нас уже есть номера кластеров для всех контекстов:

In [9]:
kmeans.labels_

array([0, 2, 2, 5, 2, 5, 2, 3, 3, 5, 1, 3, 1, 1, 5, 4, 4, 1, 3, 2, 5, 3, 1,
       1, 2, 3, 5, 1, 1, 2, 3, 2, 1, 1, 3, 3, 3, 5, 1, 0, 3, 3, 2, 2, 4, 5,
       3, 2, 0, 2, 2, 5, 5, 4, 3, 2, 4, 2, 3, 3, 3, 3, 2, 5, 2, 1, 2, 2, 0,
       5, 5, 3, 2, 3, 0, 2, 2, 2, 3, 1, 3, 2, 4, 2, 4, 3, 1, 3, 1, 3, 2, 4,
       2, 5, 5, 2, 1, 3, 1, 2, 2, 5, 1, 1, 2, 3, 5, 3, 5, 3, 2, 3, 2, 1, 2,
       0, 3, 2, 5, 2, 1, 1, 4, 4, 2, 3, 0, 3, 4, 0, 5, 2, 5, 2, 1, 1, 5, 2,
       2, 2, 3, 5, 5, 1, 3, 1, 1, 3, 5, 1, 5, 5, 1, 3, 5, 3, 2, 2, 4, 5, 5,
       3, 5, 2, 2, 2, 4, 4, 5, 3, 1, 1, 5, 2, 1, 3, 0, 3, 2, 1, 3, 5, 2, 5,
       4, 4, 2, 5, 3, 1, 1, 1, 5, 1, 2, 3, 1, 1, 2, 5, 5, 5, 5, 4, 3, 2, 2,
       2, 2, 1, 1, 1, 3, 2, 1, 5, 2, 2, 5, 3, 3, 5, 1, 0, 4, 3, 1, 3, 2, 5,
       5, 5, 2, 5, 5, 5, 1, 2, 2, 4, 2, 2, 5, 2, 4, 1, 1, 5, 3, 5, 3, 2, 5,
       1, 5, 2, 2, 1, 3, 1, 3, 1, 1, 4, 0, 1, 1, 2, 3, 5, 1, 2, 3, 3, 0, 3,
       2, 2, 5, 3, 2, 5, 2, 3, 3, 3, 2, 2, 1, 1, 2, 5, 5, 2, 5, 1, 4, 5, 3,
       2, 5,

Как оценить точность кластеризации?
Мы не можем просто сравнить номера размеченных и найденных значений - мы можем только сравнить,
насколько кластеризация похожа на "истинную":

In [10]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(ys, kmeans.labels_)

0.16039559772808296

Задания:

Непонятно, что за значения выделил метод, насколько они осмысленные?
Попробуйте представить выделенные значения:
 - через ближайшие слова
 - через типичные контексты
 
Как еще можно визуализировать работу метода и качество представлений значений? Попробуйте использовать t-SNE.